In [2]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [16]:
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2', overrides = overrides, n_seeds=10)
dispatch.dispatch_experiments('feb22', base_job_name='infocompare', game_name='large_game_2_hide', overrides = overrides, n_seeds=10)

Submitted batch job 1257
Submitted batch job 1258
Submitted batch job 1259
Submitted batch job 1260
Submitted batch job 1261
Submitted batch job 1262
Submitted batch job 1263
Submitted batch job 1264
Submitted batch job 1265
Submitted batch job 1266
Submitted batch job 1267
Dispatched 11 experiments!
Submitted batch job 1268
Submitted batch job 1269
Submitted batch job 1270
Submitted batch job 1271
Submitted batch job 1272
Submitted batch job 1273
Submitted batch job 1274
Submitted batch job 1275
Submitted batch job 1276
Submitted batch job 1277
Submitted batch job 1278
Dispatched 11 experiments!


In [19]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "_mlp  R" | awk '{print $1}' | xargs scancel

In [93]:
!squeue --format="%.18i %.65j %.2t %.10M %.6D %R"

             JOBID                                                              NAME ST       TIME  NODES NODELIST(REASON)
              2384                      br_0_minitest_tiny-feb2lstm-100_500_feb2_mlp PD       0:00      1 (Resources)
              2385                     br_0_minitest_tiny-feb2lstm-100_500_feb2_lstm PD       0:00      1 (Priority)
              2386             eval_tiny-feb2lstm-100_500_minitest_straightforward_0 PD       0:00      1 (Priority)
              2387                      br_1_minitest_tiny-feb2lstm-100_500_feb2_mlp PD       0:00      1 (Priority)
              2388                     br_1_minitest_tiny-feb2lstm-100_500_feb2_lstm PD       0:00      1 (Priority)
              2389             eval_tiny-feb2lstm-100_500_minitest_straightforward_1 PD       0:00      1 (Priority)
              2390                               eval_tiny-feb2lstm-100_500_minitest PD       0:00      1 (Priority)
              2391                     br_0_minitest_tiny

In [79]:
# !scancel 5306 5307

In [83]:
!scancel -u ubuntu

In [89]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

Submitted batch job 2354
Submitted batch job 2355
